In [4]:
import cv2
import numpy as np
import tkinter as tk
from tkinter import messagebox
from PIL import Image, ImageTk
from tensorflow.keras.models import load_model
import spotipy
from spotipy.oauth2 import SpotifyOAuth
import webbrowser

# --- Emotion Detection Setup ---
emotion_model = load_model("my_emotion_model")  # loading our model
emotion_labels = ['angry', 'happy', 'neutral', 'sad']

# --- Spotify Setup ---
client_id = '4f7d3619415b4aa2aeb7292b4fee5e54'
client_secret = '1848e80fc07a46709d569a006a964a96'
redirect_uri = 'http://localhost:8080'
scope = "user-library-read playlist-modify-public"

auth_manager = SpotifyOAuth(client_id=client_id,
                            client_secret=client_secret,
                            redirect_uri=redirect_uri,
                            scope=scope)
sp = spotipy.Spotify(auth_manager=auth_manager)

# --- Helper Functions ---
def detect_emotion(gray_frame):
    """Detects emotion from a grayscale frame"""
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    faces = face_cascade.detectMultiScale(gray_frame, 1.3, 5)

    for (x, y, w, h) in faces:
        roi_gray = gray_frame[y:y+h, x:x+w]
        roi_gray = cv2.resize(roi_gray, (48, 48), interpolation=cv2.INTER_AREA)

        if np.sum([roi_gray]) != 0:
            roi = roi_gray.astype('float') / 255.0
            roi = np.expand_dims(roi, axis=0)
            prediction = emotion_model.predict(roi)[0]
            return emotion_labels[prediction.argmax()]

    return None  # No face detected

def get_spotify_recommendations(mood):
    """Gets recommendations from Spotify based on the detected mood"""
    mood_map = {
        'angry': ['aggressive'],  
        'happy': ['pop'],  
        'neutral': ['chill'],  
        'sad': ['mellow']  
    }
    mapped_moods = mood_map.get(mood, ['happy'])  # Default to 'upbeat' for unknown mood

    results = []
    for genre in mapped_moods:
        genre_results = sp.search(q=genre, type='track', limit=5)
        results.extend(genre_results['tracks']['items'])

    recommendations = []
    for item in results:
        track_name = item['name']
        artist = item['artists'][0]['name']
        color = '#' + '%06x' % np.random.randint(0, 0xFFFFFF)  # Generate random color
        recommendations.append({'text': f'{track_name} by {artist}', 'color': color})

    return recommendations

# --- Main Program ---
class EmotionDetectorApp:
    def __init__(self, master):
        self.master = master
        master.title("Emotion Detector")
        master.configure(background='#FF0266')  # Set background color
        self.font_style = ('Inter', 12)  # Set font style

        self.video_capture = cv2.VideoCapture(0)

        self.video_frame = tk.Frame(master, bg='black', bd=2)  # Set frame color to black and increase thickness
        self.video_frame.pack(padx=10, pady=10)

        self.canvas = tk.Canvas(self.video_frame, width=self.video_capture.get(3), height=self.video_capture.get(4), bg='black')
        self.canvas.pack()

        self.emotion_label = tk.Label(master, text="Detected Emotion:", bg='#FF0266', fg='white', font=self.font_style)
        self.emotion_label.pack()

        self.recommendations_label = tk.Label(master, text="Spotify Recommendations:", bg='#FF0266', fg='white', font=self.font_style)
        self.recommendations_label.pack()

        self.detect_emotion_button = tk.Button(master, text="Detect Emotion", command=self.delayed_detect_emotion, bg='#008080', fg='white', font=self.font_style)
        self.detect_emotion_button.pack()

        self.recommendations = []

    def delayed_detect_emotion(self):
        self.display_video_frame()
        self.master.after(3000, self.detect_emotion)  # Delayed emotion detection after 3 seconds

    def detect_emotion(self):
        ret, frame = self.video_capture.read()
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        detected_emotion = detect_emotion(gray)

        if detected_emotion:
            self.emotion_label.config(text=f"Detected Emotion: {detected_emotion}")
            self.recommendations = get_spotify_recommendations(detected_emotion)

            if self.recommendations:
                self.recommendations_label.config(text="Spotify Recommendations:")
                for rec in self.recommendations:
                    recommendation_frame = tk.Frame(self.master, bg='#FF0266')
                    recommendation_frame.pack(fill=tk.X, padx=10, pady=5)

                    label = tk.Label(recommendation_frame, text=rec['text'], bg='#FF0266', fg='black', font=('Inter', 10))  # Changed fg to black
                    label.pack(side=tk.LEFT, padx=(0, 10))

                    play_button = tk.Button(recommendation_frame, text="▶", bg='#FF0266', fg='black', font=('Inter', 12),
                                            command=lambda url=self.get_spotify_track_url(rec['text']): self.open_spotify_link(url))
                    play_button.pack(side=tk.RIGHT)
            else:
                messagebox.showinfo("No Recommendations", "No recommendations found for this mood.")

        else:
            messagebox.showinfo("No Face Detected", "No face detected.")

    def display_video_frame(self):
        ret, frame = self.video_capture.read()
        if ret:
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frame = Image.fromarray(frame)
            frame = ImageTk.PhotoImage(image=frame)
            self.canvas.create_image(0, 0, anchor=tk.NW, image=frame)
            self.canvas.image = frame
            self.master.after(10, self.display_video_frame)

    def open_spotify_link(self, url):
        webbrowser.open(url)

    def get_spotify_track_url(self, track_name):
        result = sp.search(q=track_name, type='track', limit=1)
        if result and result['tracks']['items']:
            track_id = result['tracks']['items'][0]['id']
            return f"https://open.spotify.com/track/{track_id}"
        return None

    def __del__(self):
        self.video_capture.release()

def main():
    root = tk.Tk()
    app = EmotionDetectorApp(root)
    root.mainloop()

if __name__ == "__main__":
    main()


1/1 [==============================] - 0s 105ms/step
